# Introduction

We recently announced our new open-source **Agents SDK**, designed to help you build agentic AI applications using a lightweight, easy-to-use package with minimal abstractions.

This cookbook demonstrates how you can leverage the Agents SDK in combination with Stripe's API to handle dispute management, a common operational challenge many businesses face. Specifically, we focus on two real-world scenarios:

1. **Company Mistake:**  
   A scenario where the company clearly made an error, such as failing to fulfill an order, where accepting the dispute the appropriate action.

2. **Customer Dispute (Final Sale):**  
   A scenario where a customer knowingly disputes a transaction despite receiving the correct item and understanding that the purchase was final sale, requiring further investigation to gather supporting evidence.

To address these scenarios, we'll introduce three distinct agents:

- **Triage Agent:**  
  Determines whether to accept or escalate a dispute based on the fulfillment status of the order.

- **Acceptance Agent:**  
  Handles clear-cut cases by automatically accepting disputes, providing concise reasoning.

- **Investigator Agent:**  
  Conducts deeper investigations into disputes by examining customer emails, verifying IP addresses, shipping addresses, and other granular details to gather the necessary evidence.

Throughout this cookbook, we’ll guide you step-by-step, illustrating how custom agentic workflows can automate dispute management and support your business operations.


# Prerequisites

Before running this cookbook, you must set up the following accounts and complete a few setup actions. These prerequisites are essential to interact with the APIs used in this project.

### 1. OpenAI Account

- **Purpose:**  
  You need an OpenAI account to access language models and use the Agents SDK featured in this cookbook.

- **Action:**  
  [Sign up for an OpenAI account](https://openai.com) if you don’t already have one. Once you have an account, create an API key by visiting the [OpenAI API Keys page](https://platform.openai.com/api-keys).

### 2. Stripe Account

- **Purpose:**  
  A Stripe account is required to simulate payment processing, manage disputes, and interact with the Stripe API as part of our demo workflow.

- **Action:**  
  Create a free Stripe account by visiting the [Stripe Signup Page](https://dashboard.stripe.com/register).

### Stripe Setup Steps

After creating your Stripe account, follow these steps to configure and prepare your account for testing:



#### 1. Configure Your API Keys

- **Locate Your API Keys:**  
  Log in to your Stripe dashboard and navigate to **Developers > API keys**.

- **Use Test Mode:**  
  Use your **Test Secret Key** for all development and testing.



#### 2. Create a New Customer Object

Use the following `curl` command to create a customer:

```bash
curl https://api.stripe.com/v1/customers \
  -u "{SECRET_KEY}:" \
  -d name="Jenny Rosen" \
  --data-urlencode email="customer1@example.com"
```

Make sure to save the customer ID returned in the response, you'll use it later.


#### 3. Install and Configure the Stripe CLI

- **Install the Stripe CLI:**  
  Follow the [Stripe CLI installation guide](https://docs.stripe.com/stripe-cli#install) to install the CLI on your system.

- **Login to Stripe:**  
  Authenticate your Stripe user account by running the following command in your terminal:

  ```bash
  stripe login
  ```

 

- **Forward Webhook Events:**  
  Forward events to your local endpoint by running:

  ```bash
  stripe listen --forward-to localhost:5000/webhook
  ```

#### 4. Create a .env file with your OpenAI API and Stripe API Keys

```
OPENAI_API_KEY=
STRIPE_SECRET_KEY=
```

### Environment Setup
First we will install the necessary dependencies, then import the libraries and write some utility functions that we will use later on.

```
pip install quart python-dotenv openai-agents stripe nest_asyncio typing_extensions
```

In [1]:
import os
import logging
import asyncio
import json
from dotenv import load_dotenv
from quart import Quart, request
from agents import Agent, Runner, function_tool  # Only import what you need
import stripe
from typing_extensions import TypedDict, Any
import nest_asyncio
nest_asyncio.apply()
# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set Stripe API key from environment variables
stripe.api_key = os.getenv("STRIPE_SECRET_KEY")

#Create the Quart Application
app = Quart(__name__)

#### Define the Webhook Endpoint
This snippet defines a webhook endpoint using Quart that listens for Stripe's `charge.dispute.created` events. It extracts key dispute details from the incoming JSON payload and logs the information for debugging. The relevant data is then passed to a triage agent using the Agents SDK to decide on the next steps.

In [2]:
@app.route("/webhook", methods=["POST"])
async def webhook():
    """
    Handle incoming Stripe webhook events.
    Specifically processes 'charge.dispute.created' events to gather relevant data
    and pass it to the triage agent for further action.
    """
    try:
        webhook_event = await request.get_json()
    except Exception as e:
        logger.error(f"Error parsing JSON from request: {e}")
        return "Bad Request", 400

    if webhook_event.get("type") == "charge.dispute.created":
        dispute_data = webhook_event.get("data", {}).get("object", {})
        relevant_data = {
            "dispute_id": dispute_data.get("id"),
            "amount": dispute_data.get("amount"),
            "due_by": dispute_data.get("evidence_details", {}).get("due_by"),
            "payment_intent": dispute_data.get("payment_intent"),
            "reason": dispute_data.get("reason"),
            "status": dispute_data.get("status"),
            "card_brand": dispute_data.get("payment_method_details", {}).get("card", {}).get("brand")
        }

        logger.info("Charge dispute created event received")
        event_str = json.dumps(relevant_data)
        logger.info(f"Relevant dispute data: {event_str}")

        # Pass the dispute data to the triage agent
        result = await Runner.run(triage_agent, input=event_str)
        logger.info(f"Triage agent result: {result.final_output}")

    return "OK", 200


#### Define Function Tools
This section defines several helper function tools that support the dispute processing workflow. 
<br>
 
- `get_zip_code` maps a given IP address to its corresponding zip code using a predefined dictionary.
- `get_order` and `get_emails` simulate external data lookups by returning order details and email records based on provided identifiers.
- `retrieve_payment_intent`  interacts with the Stripe API to fetch payment intent details.
- `close_dispute`  automatically closes a Stripe dispute using the provided dispute ID, ensuring that disputes are properly resolved and logged.


In [3]:
@function_tool
def get_zip_code(ip_address: str) -> str:
    """
    Return the zip code corresponding to the given IP address.
    If the IP is not found, return 'Zip code not found'.
    """
    ip_to_zip = {
        "192.168.1.1": "10001",
        "192.168.1.2": "10002",
        "192.168.1.3": "10003",
        "10.0.0.1": "20001",
        "10.0.0.2": "20002"
    }
    return ip_to_zip.get(ip_address, "Zip code not found")


@function_tool
def get_order(order_id: int) -> str:
    """
    Retrieve an order by ID from a predefined list of orders.
    Returns the corresponding order object or 'No order found'.
    """
    orders = [
        {
            "order_id": 1234,
            "fulfillment_details": "not_shipped"
        },
        {
            "order_id": 9101,
            "fulfillment_details": "shipped",
            "tracking_info": {
                "carrier": "FedEx",
                "tracking_number": "123456789012"
            },
            "delivery_status": "out for delivery"
        },
        {
            "order_id": 1121,
            "fulfillment_details": "delivered",
            "customer_id": "cus_PZ1234567890",
            "order_date": "2023-01-01",
            "customer_email": "customer1@example.com",
            "tracking_info": {
                "carrier": "UPS",
                "tracking_number": "1Z999AA10123456784",
                "delivery_status": "delivered"
            },
            "shipping_address": {
                "zip": "10001"
            },
            "tos_acceptance": {
                "date": "2023-01-01",
                "ip": "192.168.1.1"
            }
        }
    ]
    for order in orders:
        if order["order_id"] == order_id:
            return order
    return "No order found"


@function_tool
def get_emails(email: str) -> list:
    """
    Return a list of email records for the given email address.
    """
    emails = [
        {
            "email": "customer1@example.com",
            "subject": "Order #1121",
            "body": "Hey, I know you don't accept refunds but the sneakers don't fit and I'd like a refund"
        },
        {
            "email": "customer2@example.com",
            "subject": "Inquiry about product availability",
            "body": "Hello, I wanted to check if the new model of the smartphone is available in stock."
        },
        {
            "email": "customer3@example.com",
            "subject": "Feedback on recent purchase",
            "body": "Hi, I recently purchased a laptop from your store and I am very satisfied with the product. Keep up the good work!"
        }
    ]
    return [email_data for email_data in emails if email_data["email"] == email]


@function_tool
async def retrieve_payment_intent(payment_intent_id: str) -> dict:
    """
    Retrieve a Stripe payment intent by ID.
    Returns the payment intent object on success or an empty dictionary on failure.
    """
    try:
        return stripe.PaymentIntent.retrieve(payment_intent_id)
    except stripe.error.StripeError as e:
        logger.error(f"Stripe error occurred while retrieving payment intent: {e}")
        return {}

@function_tool
async def close_dispute(dispute_id: str) -> dict:
    """
    Close a Stripe dispute by ID. 
    Returns the dispute object on success or an empty dictionary on failure.
    """
    try:
        return stripe.Dispute.close(dispute_id)
    except stripe.error.StripeError as e:
        logger.error(f"Stripe error occurred while closing dispute: {e}")
        return {}


### Define the Agents

- The **Dispute Intake Agent (investigator_agent)** is responsible for investigating disputes by gathering all relevant evidence, such as retrieving customer emails and verifying that the IP address used for TOS acceptance matches the shipping address zip code.
- The **Accept a Dispute Agent (accept_dispute_agent)** handles disputes that are determined to be valid by automatically closing them and providing a brief explanation for the decision.
- The **Triage Agent (triage_agent)** serves as the decision-maker by extracting the order ID from the payment intent's metadata, retrieving detailed order information, and then deciding whether to escalate the dispute to the investigator or to pass it to the accept dispute agent.
- Together, these agents form a modular workflow that automates and streamlines the dispute resolution process by delegating specific tasks to specialized agents.


In [4]:
investigator_agent = Agent(
    name="Dispute Intake Agent",
    instructions=(
        "In your role as a dispute investigator, thoroughly investigate the dispute, gather all relevant evidence, "
        "identify any customer emails and provide their body text, and confirm that the IP address used to "
        "accept the Terms of Service matches the shipping address zip code."
    ),
    tools=[get_emails, get_zip_code]
)

accept_dispute_agent = Agent(
    name="Accept a Dispute",
    instructions="Close the dispute using the given id. Provide a short explanation, for example: 'I accepted this dispute because the order hasn't been fulfilled yet with a reference to the order that was retrieved from the database.'",
    tools=[close_dispute]
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Find the order id in the payment intent's metadata, retrieve more details about the order, and if "
                 "the order was shipped, escalate to the investigator agent. Otherwise, accept the dispute.",
    tools=[retrieve_payment_intent, get_order],
    handoffs=[accept_dispute_agent, investigator_agent],
)

## Running the Server and Simulating Payments

The snippet below starts the Quart web server that listens for incoming Stripe webhook events. Since the webhook is waiting for an event to be fired, you need to create a new payment with a test card for the customer to simulate a dispute scenario. 

**Request 1:**  
  This command simulates a payment intent where the customer disputes because they claim they never received the product, and it is an order that wasn't shipped. In this case, the system should accept the dispute.

```
curl https://api.stripe.com/v1/payment_intents \
  -u "{SECRET_KEY}:" \
  -d amount=2000 \
  -d currency=usd \
  -d customer={CUSTOMER_OBJECT_ID} \
  -d payment_method=pm_card_createDisputeProductNotReceived \
  -d confirm=true \
  -d off_session=true \
  -d "metadata[order_id]"=1234 \
  -d "automatic_payment_methods[enabled]"=true

```

**Request 2:**  
  This command simulates a payment intent where the customer, despite knowing the return policy is final sale, disputes the charge. This scenario requires further investigation to gather enough evidence for a proper response.


```
curl https://api.stripe.com/v1/payment_intents \
  -u "{STRIPE_SECRET_KEY}:" \
  -d amount=2000 \
  -d currency=usd \
  -d customer={CUSTOMER_ID} \
  -d payment_method=pm_card_createDispute \
  -d confirm=true \
  -d off_session=true \
  -d "metadata[order_id]"=1121 \
  -d "automatic_payment_methods[enabled]"=true

```

Run the above requests to simulate payment events and trigger the complete agent-based workflow.

In [ ]:
def run_server():
    """
    Run the Quart web server.
    """
    app.run(host='0.0.0.0', port=5000)

# Uncomment the following line to run the server directly from the notebook.
run_server()


[2025-03-17 17:30:05 -0400] [24572] [INFO] Running on http://0.0.0.0:5000 (CTRL + C to quit)
INFO:hypercorn.error:Running on http://0.0.0.0:5000 (CTRL + C to quit)


 * Serving Quart app '__main__'
 * Debug mode: False
 * Please use an ASGI server (e.g. Hypercorn) directly in production
 * Running on http://0.0.0.0:5000 (CTRL + C to quit)
[2025-03-17 17:30:09 -0400] [24572] [INFO] 127.0.0.1:58187 POST /webhook 1.1 200 2 2598
[2025-03-17 17:30:10 -0400] [24572] [INFO] 127.0.0.1:58187 POST /webhook 1.1 200 2 1878
[2025-03-17 17:30:10 -0400] [24572] [INFO] 127.0.0.1:58187 POST /webhook 1.1 200 2 2185


INFO:__main__:Charge dispute created event received
INFO:__main__:Relevant dispute data: {"dispute_id": "dp_1R3lEbIHWlrjZQHrOxiLtA9E", "amount": 2000, "due_by": 1743033599, "payment_intent": "pi_3R3lEaIHWlrjZQHr1TFQ9QgZ", "reason": "fraudulent", "status": "needs_response", "card_brand": "visa"}


[2025-03-17 17:30:10 -0400] [24572] [INFO] 127.0.0.1:58190 POST /webhook 1.1 200 2 1258
[2025-03-17 17:30:12 -0400] [24572] [INFO] 127.0.0.1:58190 POST /webhook 1.1 200 2 1465


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:stripe:message='Request to Stripe api' method=get url=https://api.stripe.com/v1/payment_intents/pi_3R3lEaIHWlrjZQHr1TFQ9QgZ
INFO:stripe:message='Stripe API response' path=https://api.stripe.com/v1/payment_intents/pi_3R3lEaIHWlrjZQHr1TFQ9QgZ response_code=200
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:__main__:Triage agent result: Here's the collected information for the dispute investigation:

### Payment Details:
- **Dispute ID**: `dp_1R3lEbIHWlrjZQHrOxiLtA9E`
- **Amount**: $2000 
- **Reason for Dispute**: Fraudulent
- **Payment Intent**: `pi_3R3lEaIHWlrjZQHr1TFQ9QgZ`
- **Status**: Needs response
- **Card

[2025-03-17 17:30:33 -0400] [24572] [INFO] 127.0.0.1:58187 POST /webhook 1.1 200 2 23679803
